# 🛍️ Mall Customer Segmentation — Complete Notebook

## What this notebook covers
1. **Data Loading & Exploration** — understand what we have  
2. **Feature Selection** — *why* we pick certain features (with proof, not guessing)  
3. **Elbow Method** — find candidate K values  
4. **Silhouette Validation** — confirm the optimal K objectively  
5. **Feature Set Comparison** — prove which features produce the best clusters  
6. **Final Clustering & Visualization** — apply KMeans with the optimal K  

> **Key principle:** Feature selection comes *before* choosing K. Get features wrong → K is meaningless.


---
## 1. Imports & Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context("notebook", font_scale=1.1)


---
## 2. Data Loading & Exploration

Before touching any model, understand your data.

In [2]:
data = pd.read_csv('Mall_Customers.csv', index_col='CustomerID')
print("Shape:", data.shape)
data.head(10)


FileNotFoundError: [Errno 2] No such file or directory: 'Mall_Customers.csv'

In [ ]:
print("── Data Types ──")
print(data.dtypes)
print("\n── Null Values ──")
print(data.isnull().sum())
print("\n── Descriptive Statistics ──")
data.describe()


In [ ]:
# Distribution of each numeric feature
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for ax, col in zip(axes, ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']):
    ax.hist(data[col], bins=25, color='steelblue', edgecolor='white', alpha=0.85)
    ax.set_title(col)
    ax.set_ylabel('Count')
plt.tight_layout()
plt.show()


In [ ]:
# Gender distribution
print(data['Gender'].value_counts())
data['Gender'].value_counts().plot(kind='bar', color=['#e07a5f','#81b29a'], edgecolor='white')
plt.title('Gender Distribution')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


---
## 3. Feature Selection — The Critical Step

> **Why this matters:** KMeans uses Euclidean distance. If a feature is noise, it increases distances randomly and *breaks* your clusters. We need to **prove** which features are useful — not guess.

### 3 tests we run:
1. **Correlation matrix** — are features redundant?  
2. **Group means by Gender** — does Gender actually separate anything?  
3. **Silhouette across feature sets** — which combination produces the tightest clusters?


### 3a. Correlation Matrix

Check if numeric features are redundant with each other.

In [ ]:
corr = data[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']].corr()

fig, ax = plt.subplots(figsize=(7, 5))
sns.heatmap(corr, annot=True, fmt='.3f', cmap='coolwarm', center=0,
            vmin=-1, vmax=1, square=True, linewidths=1, cbar_kws={'shrink': 0.8}, ax=ax)
ax.set_title('Correlation Matrix', fontsize=13, pad=12)
plt.tight_layout()
plt.show()

print("\n── Interpretation ──")
print(f"Age  <-> Income:         {corr.loc['Age','Annual Income (k$)']:.3f}  → almost zero")
print(f"Age  <-> Spending Score: {corr.loc['Age','Spending Score (1-100)']:.3f}  → weak negative")
print(f"Income <-> Spending:     {corr.loc['Annual Income (k$)','Spending Score (1-100)']:.3f}  → independent")


### 3b. Does Gender Separate the Data?

If mean values are nearly identical across genders, Gender carries **no clustering signal** for KMeans.

In [ ]:
print("── Mean by Gender ──")
print(data.groupby('Gender')[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']].mean().round(2))
print("\n── Std by Gender ──")
print(data.groupby('Gender')[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']].std().round(2))

# Visual comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
cols = ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']
for ax, col in zip(axes, cols):
    data.groupby('Gender')[col].mean().plot(kind='bar', ax=ax, color=['#e07a5f','#81b29a'], edgecolor='white')
    ax.set_title(f'Mean {col} by Gender')
    ax.set_ylabel('Mean')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
plt.tight_layout()
plt.show()

print("\n→ Conclusion: Differences are tiny. Gender adds no meaningful signal for KMeans.")


### 3c. Silhouette Score Across Feature Sets

This is the **definitive test**. Run KMeans with the same K on 3 different feature sets and compare silhouette scores. Highest score = best features.

In [ ]:
scaler = StandardScaler()
encoder = LabelEncoder()

# Feature Set A: Income + Spending only
X_a = scaler.fit_transform(data[['Annual Income (k$)', 'Spending Score (1-100)']])

# Feature Set B: Age + Income + Spending
X_b = scaler.fit_transform(data[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']])

# Feature Set C: All 4 (including encoded Gender)
data_c = data.copy()
data_c['Gender_enc'] = encoder.fit_transform(data_c['Gender'])
X_c = scaler.fit_transform(data_c[['Age', 'Annual Income (k$)', 'Spending Score (1-100)', 'Gender_enc']])

feature_sets = {
    'Income + Spending':          X_a,
    'Age + Income + Spending':    X_b,
    'All 4 (incl. Gender)':       X_c,
}

# Compare silhouette for K = 2..10
k_range = range(2, 11)
results = {name: [] for name in feature_sets}

for name, X in feature_sets.items():
    for k in k_range:
        km = KMeans(n_clusters=k, n_init=10, random_state=42)
        labels = km.fit_predict(X)
        results[name].append(silhouette_score(X, labels))

# ── Plot ──
fig, ax = plt.subplots(figsize=(10, 5))
colors_map = {'Income + Spending': '#e07a5f',
              'Age + Income + Spending': '#81b29a',
              'All 4 (incl. Gender)': '#5b8db8'}

for name, scores in results.items():
    ax.plot(list(k_range), scores, marker='o', linewidth=2.2, label=name, color=colors_map[name])

ax.set_xlabel('Number of Clusters (K)', fontsize=12)
ax.set_ylabel('Silhouette Score', fontsize=12)
ax.set_title('Silhouette Score — Feature Set Comparison', fontsize=14)
ax.legend(fontsize=11)
ax.set_xticks(list(k_range))
ax.grid(True, alpha=0.4)
plt.tight_layout()
plt.show()

# ── Print table ──
print("\n── Silhouette Scores ──\n")
header = f"{'K':<4}" + "".join(f"{name:<32}" for name in feature_sets)
print(header)
print("─" * len(header))
for i, k in enumerate(k_range):
    row = f"{k:<4}"
    vals = [results[name][i] for name in feature_sets]
    best = max(vals)
    for name in feature_sets:
        v = results[name][i]
        tag = " ◀ best" if v == best else ""
        row += f"{v:.4f}{tag:<25}"
    print(row)


> **Conclusion:** `Income + Spending` wins at every K.  
> Age adds noise. Gender adds more noise.  
> We use **only Income and Spending Score** going forward.

---
## 4. Elbow Method — Finding the Candidate K

**What it does:** Plot WCSS vs K. The "elbow" (sharpest bend) is the candidate K.  
**Limitation:** Subjective — the elbow can be ambiguous. That's why we confirm with Silhouette.  
**Important:** Always start from K = 1. Starting at K = 15 (like in your original notebook) means you've already passed the elbow.


In [ ]:
# Use the proven best features
X = scaler.fit_transform(data[['Annual Income (k$)', 'Spending Score (1-100)']])

K_RANGE = range(1, 11)
wcss = []

for k in K_RANGE:
    km = KMeans(n_clusters=k, init='k-means++', n_init=10, random_state=42)
    km.fit(X)
    wcss.append(km.inertia_)

# ── Programmatic elbow detection: max perpendicular distance from line ──
p1 = np.array([1, wcss[0]])
p2 = np.array([10, wcss[-1]])
line_vec  = p2 - p1
line_unit = line_vec / np.linalg.norm(line_vec)

distances = []
for i, k in enumerate(K_RANGE):
    p    = np.array([k, wcss[i]])
    proj = np.dot(p - p1, line_unit) * line_unit
    distances.append(np.linalg.norm((p - p1) - proj))

optimal_k_elbow = list(K_RANGE)[np.argmax(distances)]

# ── Plot ──
fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(list(K_RANGE), wcss, 'o-', color='steelblue', linewidth=2.5,
        markersize=8, markerfacecolor='mediumpurple', markeredgecolor='white', markeredgewidth=1.5)

ax.plot(optimal_k_elbow, wcss[optimal_k_elbow - 1], 'o', color='#e07a5f', markersize=16,
        markeredgecolor='white', markeredgewidth=2.5, zorder=5)
ax.annotate(f'Elbow at K = {optimal_k_elbow}',
            xy=(optimal_k_elbow, wcss[optimal_k_elbow - 1]),
            xytext=(optimal_k_elbow + 1.3, wcss[optimal_k_elbow - 1] + 30),
            fontsize=13, fontweight='bold', color='#e07a5f',
            arrowprops=dict(arrowstyle='->', color='#e07a5f', lw=2))
ax.axvline(optimal_k_elbow, color='#e07a5f', linestyle='--', alpha=0.5)

ax.set_xlabel('Number of Clusters (K)', fontsize=13)
ax.set_ylabel('WCSS', fontsize=13)
ax.set_title('Elbow Method', fontsize=15)
ax.set_xticks(list(K_RANGE))
ax.grid(True, alpha=0.4)
plt.tight_layout()
plt.show()

print(f"\nElbow detected at K = {optimal_k_elbow}")
print("\n── WCSS Table ──")
for k, w in zip(K_RANGE, wcss):
    print(f"  K={k:2d}  →  WCSS = {w:8.2f}" + ("  ◀ Elbow" if k == optimal_k_elbow else ""))


---
## 5. Silhouette Score — Confirming the Optimal K

**What it measures:** How similar each point is to its own cluster vs the nearest other cluster.  
- **+1** → well separated  
- **0** → on the boundary  
- **−1** → wrong cluster  

Pick the K with the **highest** average silhouette. This is the objective confirmation step.


In [ ]:
sil_scores = []
k_range_sil = range(2, 11)   # silhouette needs K >= 2

for k in k_range_sil:
    km = KMeans(n_clusters=k, n_init=10, random_state=42)
    labels = km.fit_predict(X)
    sil_scores.append(silhouette_score(X, labels))

optimal_k_sil = list(k_range_sil)[np.argmax(sil_scores)]

# ── Plot ──
fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(list(k_range_sil), sil_scores, 'o-', color='#81b29a', linewidth=2.5,
        markersize=8, markerfacecolor='#e07a5f', markeredgecolor='white', markeredgewidth=1.5)

ax.plot(optimal_k_sil, max(sil_scores), 'o', color='#e07a5f', markersize=16,
        markeredgecolor='white', markeredgewidth=2.5, zorder=5)
ax.annotate(f'Best: K = {optimal_k_sil}  (score = {max(sil_scores):.4f})',
            xy=(optimal_k_sil, max(sil_scores)),
            xytext=(optimal_k_sil + 1.2, max(sil_scores) - 0.04),
            fontsize=12, fontweight='bold', color='#e07a5f',
            arrowprops=dict(arrowstyle='->', color='#e07a5f', lw=2))
ax.axvline(optimal_k_sil, color='#e07a5f', linestyle='--', alpha=0.5)

ax.set_xlabel('Number of Clusters (K)', fontsize=13)
ax.set_ylabel('Silhouette Score', fontsize=13)
ax.set_title('Silhouette Score Validation', fontsize=15)
ax.set_xticks(list(k_range_sil))
ax.grid(True, alpha=0.4)
plt.tight_layout()
plt.show()

print(f"\nElbow method said:      K = {optimal_k_elbow}")
print(f"Silhouette method says: K = {optimal_k_sil}")
print(f"\n→ Both agree. We use K = {optimal_k_sil}.")


---
## 6. Final Clustering & Visualization

Apply KMeans with the confirmed K and visualize the customer segments.

In [ ]:
K_FINAL = optimal_k_sil
km_final = KMeans(n_clusters=K_FINAL, n_init=10, random_state=42)
data['Cluster'] = km_final.fit_predict(X)

# ── Scatter plot ──
fig, ax = plt.subplots(figsize=(10, 7))
palette = plt.cm.tab10.colors

for cid in range(K_FINAL):
    mask = data['Cluster'] == cid
    ax.scatter(data.loc[mask, 'Annual Income (k$)'],
               data.loc[mask, 'Spending Score (1-100)'],
               c=[palette[cid]], s=70, edgecolors='white', linewidths=1,
               label=f'Cluster {cid}', alpha=0.85)

# Centroids (inverse-scaled)
scaler_final = StandardScaler()
scaler_final.fit(data[['Annual Income (k$)', 'Spending Score (1-100)']])
centroids_orig = scaler_final.inverse_transform(km_final.cluster_centers_)

ax.scatter(centroids_orig[:, 0], centroids_orig[:, 1],
           c='black', s=200, marker='X', edgecolors='white', linewidths=2, zorder=5, label='Centroids')

ax.set_xlabel('Annual Income (k$)', fontsize=13)
ax.set_ylabel('Spending Score (1-100)', fontsize=13)
ax.set_title(f'Mall Customer Segmentation  (K = {K_FINAL})', fontsize=15)
ax.legend(fontsize=11, loc='upper left')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


### Cluster Profiles — Who is in each segment?

In [ ]:
profile = data.groupby('Cluster').agg(
    Count          = ('Age', 'size'),
    Avg_Age        = ('Age', 'mean'),
    Avg_Income     = ('Annual Income (k$)', 'mean'),
    Avg_Spending   = ('Spending Score (1-100)', 'mean'),
    Female_Pct     = ('Gender', lambda x: round((x == 'Female').mean() * 100, 1))
).round(2)

print("── Cluster Profiles ──\n")
print(profile.to_string())


In [ ]:
# Bar charts per metric
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
metrics = ['Avg_Income', 'Avg_Spending', 'Avg_Age']
titles  = ['Average Income (k$)', 'Average Spending Score', 'Average Age']

for ax, col, title in zip(axes, metrics, titles):
    bars = ax.bar(profile.index, profile[col],
                  color=[palette[i] for i in range(K_FINAL)], edgecolor='white')
    ax.set_title(title, fontsize=13)
    ax.set_xlabel('Cluster')
    ax.set_ylabel(title)
    ax.set_xticks(range(K_FINAL))
    for bar in bars:
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{bar.get_height():.1f}', ha='center', fontsize=10, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('Cluster Profile Comparison', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()


---
## 7. Summary — What We Learned & Why

| Step | What we did | Why |
|---|---|---|
| Correlation check | Checked pairwise correlations | Catch redundant features early |
| Gender test | Compared means by gender | KMeans is distance-based — if groups look the same, the feature is useless |
| Silhouette across feature sets | Ran KMeans on 3 feature combos | **Objective proof** of which features produce best clusters |
| Elbow Method | Plotted WCSS from K=1 to 10 | Get a visual candidate for K — but it's subjective |
| Silhouette Validation | Plotted silhouette from K=2 to 10 | **Objective confirmation** of the optimal K |
| Final model | KMeans with confirmed K on confirmed features | Everything is data-driven, nothing guessed |

### When to use what:
- **Elbow** → first pass, gives you a candidate range  
- **Silhouette** → second pass, confirms the best K objectively  
- **If they disagree** → trust Silhouette  
- **Feature selection** → always validate with silhouette across feature sets before you commit
